In [17]:
# Helper functions
def nshot_chats(nshot_data, n, question):
    """Generate n-shot prompt with the specified number of examples."""
    def question_prompt(s):
        return f"Question: {s}"
    
    def answer_prompt(s):
        return f"Answer: {s}"

    # Prepare few-shot examples
    chats = []
    random.seed(42)  # for reproducibility
    for qna in random.sample(list(nshot_data), n):
        chats.append({"role": "user", "content": question_prompt(qna["question"])})
        chats.append({"role": "assistant", "content": answer_prompt(qna["answer"])})
    
    # Add the target question
    chats.append({"role": "user", "content": question_prompt(question) + "\nLet's think step by step. At the end, you MUST write the answer as an integer after '####'."})
    return chats

def extract_ans_from_response(answer):
    """Extracts both the prelude (before '####') and the answer (after '####' up to newline) from the model's response."""
    
    marker = "####"
    # Locate the marker in the text
    start_index = answer.find(marker)
    
    if start_index != -1:
        # Separate the portion before the marker
        reasoning = answer[:start_index].strip()
        
        # Move start_index to the end of the marker
        start_index += len(marker)
        
        # Find the position of the next newline after the marker
        end_index = answer.find("\n", start_index)
        if end_index == -1:
            # If there's no newline, grab until the end of the text
            end_index = len(answer)
        
        # Extract the text after the marker up to the newline
        extracted_answer = answer[start_index:end_index].strip()
    else:
        # If '####' marker is not found, treat the entire answer as the prelude
        reasoning = answer.strip()
        extracted_answer = None
        print("Marker '####' not found in response.")
    
    # Clean up the extracted answer (remove commas, symbols, etc.)
    if extracted_answer:
        for remove_char in [',', '$', '%', 'g']:
            extracted_answer = extracted_answer.replace(remove_char, '')
        
        # Try to convert the cleaned answer to an integer if possible
        try:
            extracted_answer = int(extracted_answer)
        except ValueError:
            pass
    
    return reasoning, extracted_answer
